In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('../public/e_use.csv')

data

# data = data.transpose() # 행 열을 돌려 반전시킨다.

# data


,분류별(1),분류별(2),1981,1982,1983,1984,1985,1986,1987,1988,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,합계 (천toe),소계,38952,38711,41337,44998,46998,50524,55197,61033,...,206974.0,208376.0,210477.0,215447.0,222027.0,231179.0,233585.0,231621.0,222563.0,237360.0
1,증가율 (%),소계,3.6,-0.6,6.8,8.9,4.4,7.5,9.2,10.6,...,0.8,0.7,1.0,2.4,3.1,4.1,1.0,-0.8,-3.9,6.6
2,석탄 (천toe),소계,14478,14548,15336,16806,17940,18887,19778,20565,...,31017.0,31603.0,35198.0,34849.0,32342.0,33360.0,32480.0,32057.0,30457.0,31810.0
3,석탄 (천toe),무연탄,9571,8937,9338,10600,11632,12336,12006,11527,...,4511.0,4649.0,4212.0,4539.0,4431.0,3986.0,4565.0,3850.0,3459.0,3577.0
4,석탄 (천toe),유연탄,4906,5612,5997,6206,6308,6552,7772,9039,...,26507.0,26954.0,30985.0,30310.0,27911.0,29374.0,27915.0,28207.0,26998.0,28233.0
5,석유 (천toe),소계,18912,18460,19921,21602,22580,25181,28372,32511,...,101683.0,101522.0,102664.0,106854.0,114264.0,118289.0,116831.0,116125.0,109344.0,116936.0
6,석유 (천toe),에너지유,15471,14835,15589,16690,17360,19211,21767,25083,...,40279.0,39424.0,39423.0,42488.0,45372.0,46364.0,45496.0,44847.0,41334.0,41912.0
7,석유 (천toe),LPG,500,735,1010,1242,1424,1736,2110,2561,...,9194.0,9536.0,9271.0,9214.0,11059.0,10579.0,10945.0,12188.0,12069.0,12034.0
8,석유 (천toe),비에너지유,2942,2890,3322,3670,3795,4235,4495,4866,...,52209.0,52563.0,53971.0,55151.0,57833.0,61346.0,60389.0,59091.0,55941.0,62990.0
9,천연가스 (천toe),소계,-,-,-,-,-,-,-,-,...,1156.0,813.0,769.0,851.0,1044.0,1370.0,2038.0,2909.0,3602.0,3550.0
